In [1]:
from pyspark import *
sc = SparkContext(master="local", appName="test")
sc

<SparkContext master=local appName=test>

In [36]:
import urllib.request as ur
f = ur.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)
raw_data
#.collect() passe la donnée en liste python

./kddcup.data_10_percent.gz MapPartitionsRDD[68] at textFile at NativeMethodAccessorImpl.java:0

In [37]:
raw_data = raw_data.map(lambda x: x.split(','))
from pyspark.sql.types import *
structure = StructType([
    StructField("duration", IntegerType(),     True),
    StructField("protocol_type", StringType(), True),
    StructField("service", StringType(),       True),
    StructField("flag", StringType(),       True),
    StructField("src_bytes", IntegerType(),    True),
    StructField("dest_bytes", IntegerType(),   True),
    StructField("interactions", StringType(),  True)
])
structure

StructType(List(StructField(duration,IntegerType,true),StructField(protocol_type,StringType,true),StructField(service,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,IntegerType,true),StructField(dest_bytes,IntegerType,true),StructField(interactions,StringType,true)))

In [49]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

pop = raw_data.map(lambda key: (int(key[0]), key[1], key[2], key[3], int(key[4]), int(key[5]), key[-1]))
df = sqlContext.createDataFrame(pop, structure)
df.createOrReplaceTempView('interactions')
df.show(10)


+--------+-------------+-------+----+---------+----------+------------+
|duration|protocol_type|service|flag|src_bytes|dest_bytes|interactions|
+--------+-------------+-------+----+---------+----------+------------+
|       0|          tcp|   http|  SF|      181|      5450|     normal.|
|       0|          tcp|   http|  SF|      239|       486|     normal.|
|       0|          tcp|   http|  SF|      235|      1337|     normal.|
|       0|          tcp|   http|  SF|      219|      1337|     normal.|
|       0|          tcp|   http|  SF|      217|      2032|     normal.|
|       0|          tcp|   http|  SF|      217|      2032|     normal.|
|       0|          tcp|   http|  SF|      212|      1940|     normal.|
|       0|          tcp|   http|  SF|      159|      4087|     normal.|
|       0|          tcp|   http|  SF|      210|       151|     normal.|
|       0|          tcp|   http|  SF|      212|       786|     normal.|
+--------+-------------+-------+----+---------+----------+------

In [52]:
res = df[(df['duration'] > 1000) & (df['protocol_type'] == 'tcp') & (df['dest_bytes'] == 0)]
res.count()

139

In [ ]:
result.rdd.map(lambda x : "Duration: {0}, Dest. bytes: {1}".format(x.dest_bytes,x.duration)).take(10)
